<a href="https://colab.research.google.com/github/2020-nlp-c/nlp-statisticsmodel/blob/master/jyp/document_summerize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [108]:
import pandas as pd
from itertools import permutations
import numpy as np

In [109]:
class textrank_sen_1:    
    def __init__(self,document):
        self.document=document
    def sen_ls(self):
        L_sen=self.document.split(". ")
        L_sen[-1]=L_sen[-1].strip(".")
        L_sen_token_ls=[x.split() for x in L_sen]

        return L_sen_token_ls

    def connected_matrix(self):
        m=len(self.sen_ls())
        Mat=np.zeros((m,m))

        for v in list(permutations(range(m),2)):
            Mat[v[0]][v[1]]=len(set(self.sen_ls()[v[0]]).intersection(set(self.sen_ls()[v[1]])))/len(set(self.sen_ls()[v[0]]).union(set(self.sen_ls()[v[1]])))
        for i,x in enumerate(Mat):
            Mat[i]=Mat[i]/sum(Mat[i])

        return Mat

    def score(self,d,num_epoch,threshold):
        m=len(self.sen_ls())
        # score=np.ones(m)*10
        score=np.random.randint(1,100,size=m)
        table=[]
        for i in range(num_epoch):
            new_score=(1-d)+np.dot(self.connected_matrix().T,d*score)
            if (np.abs(score-new_score)).mean()<threshold:
                break
            score=new_score
        for i,x in enumerate(self.sen_ls()):
            sen=' '.join(x)
            table.append({"sentence":sen,"score":new_score[i]})
        table=pd.DataFrame(table)
        return table

In [110]:
class textrank_sen_2:    
    def __init__(self,document):
        self.document=document
    def sen_ls(self):
        L_sen=self.document.split(". ")
        L_sen[-1]=L_sen[-1].strip(".")
        L_sen_token_ls=[x.split() for x in L_sen]

        return L_sen_token_ls

    def connected_matrix(self):
        m=len(self.sen_ls())
        Mat=np.zeros((m,m))

        for v in list(permutations(range(m),2)):
            Mat[v[0]][v[1]]=len(set(self.sen_ls()[v[0]]).intersection(set(self.sen_ls()[v[1]])))/len(set(self.sen_ls()[v[0]]).union(set(self.sen_ls()[v[1]])))

        first_score=np.array([sum(Mat[i]) for i,_ in enumerate(Mat)])

        for i,x in enumerate(Mat):
            
            Mat[i]=Mat[i]/sum(Mat[i])

        return Mat,first_score

    def score(self,d,num_epoch,threshold):
        m=len(self.sen_ls())

        # score=np.ones(m)
        table=[]
        connected_matrix,score=self.connected_matrix()
        for i in range(num_epoch):
            new_score=(1-d)+np.dot(connected_matrix.T,d*score)
            if (np.abs(score-new_score)).mean()<threshold:
                break
            score=new_score
        for i,x in enumerate(self.sen_ls()):
            sen=' '.join(x)
            table.append({"sentence":sen,"score":new_score[i]})
        table=pd.DataFrame(table)
        return table

In [111]:
document="딸기 바나나 사과 파인애플 수박. 바나나 사과 딸기 포도. 복숭아 수박. 파인애플 사과 딸기 바나나."

In [112]:
textrank_sen_1(document).score(d=0.85,num_epoch=1000,threshold=0.0001)

,sentence,score
0,딸기 바나나 사과 파인애플 수박,1.411018
1,바나나 사과 딸기 포도,1.025079
2,복숭아 수박,0.286303
3,파인애플 사과 딸기 바나나,1.279563


In [113]:
textrank_sen_2(document).score(d=0.85,num_epoch=1000,threshold=0.0001)

,sentence,score
0,딸기 바나나 사과 파인애플 수박,1.411013
1,바나나 사과 딸기 포도,1.025076
2,복숭아 수박,0.286303
3,파인애플 사과 딸기 바나나,1.279558
